In [9]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

In [10]:
data = pd.read_csv("D:/nckh/ML/cleaned_ecommerce_data.csv")
data = data.drop(['Purchase Year','Purchase Month','Churn'],axis=1)

In [12]:
data['Purchase Date'] = pd.to_datetime(data['Purchase Date'])
data['Produc tCategory'] = data['Product Category'].astype('category')
data['Gender'] = data['Gender'].astype('category')

In [13]:
le_category = LabelEncoder()
data["Product Category"] = le_category.fit_transform(data["Product Category"])
ohe_gender = OneHotEncoder(sparse=False, handle_unknown='ignore')
data_encoded = ohe_gender.fit_transform(data[["Gender"]])
data = pd.concat([data.drop(["Gender"], axis=1), pd.DataFrame(data_encoded, columns=["Female", "Male"])], axis=1)

data

c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Customer ID,Purchase Date,Product Category,Product Price,Quantity,Total Purchase Amount,Customer Age,Produc tCategory,Female,Male
0,44605,2023-05-03,3,177,1,177,31,Home,1.0,0.0
1,44605,2021-05-16,2,174,3,522,31,Electronics,1.0,0.0
2,44605,2020-07-13,0,413,1,413,31,Books,1.0,0.0
3,44605,2023-01-17,2,396,3,1188,31,Electronics,1.0,0.0
4,44605,2021-05-01,0,259,4,1036,31,Books,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
249995,33807,2023-01-24,3,436,1,436,63,Home,0.0,1.0
249996,20455,2021-06-04,2,233,1,233,66,Electronics,1.0,0.0
249997,28055,2022-11-10,2,441,5,2205,63,Electronics,1.0,0.0
249998,15023,2021-06-27,2,44,2,88,64,Electronics,0.0,1.0


In [14]:
data["Recency"] = (pd.to_datetime("today") - data["Purchase Date"]).dt.days
data["NumOrders"] = data.groupby("Customer ID")["Purchase Date"].transform("size")
data["AvgOrderValue"] = data.groupby("Customer ID")["Total Purchase Amount"].transform("mean")
data["TopProduct"] = data.groupby("Customer ID")["Product Category"].transform(lambda x: x.value_counts().index[0])


In [15]:
features = ["Recency", "NumOrders", "AvgOrderValue", "TopProduct", "Female", "Male"]
target = "Product Category"

In [16]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.25)

In [17]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Độ chính xác mô hình Random Forest:", accuracy_rf)

Độ chính xác mô hình Random Forest: 0.37192


In [ ]:
data

,Customer ID,Purchase Date,Product Category,Product Price,Quantity,Total Purchase Amount,Customer Age,Female,Male,Recency,NumOrders,AvgOrderValue,TopProduct
0,44605,2023-05-03,3,177,1,177,31,1.0,0.0,284,5,667.2,2
1,44605,2021-05-16,2,174,3,522,31,1.0,0.0,1001,5,667.2,2
2,44605,2020-07-13,0,413,1,413,31,1.0,0.0,1308,5,667.2,2
3,44605,2023-01-17,2,396,3,1188,31,1.0,0.0,390,5,667.2,2
4,44605,2021-05-01,0,259,4,1036,31,1.0,0.0,1016,5,667.2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,33807,2023-01-24,3,436,1,436,63,0.0,1.0,383,1,436.0,3
249996,20455,2021-06-04,2,233,1,233,66,1.0,0.0,982,1,233.0,2
249997,28055,2022-11-10,2,441,5,2205,63,1.0,0.0,458,1,2205.0,2
249998,15023,2021-06-27,2,44,2,88,64,0.0,1.0,959,1,88.0,2


In [20]:
data_customer = data[data["Customer ID"] == 44605]
prediction = rf.predict(data_customer[features].iloc[-1,:].to_numpy().reshape(1, -1))


# In kết quả dự đoán
print(f"Ngành hàng dự đoán: {le_category.inverse_transform([prediction])[0]}")

Ngành hàng dự đoán: Electronics


c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
